# Training Models Baselines

In [16]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

In [43]:
met0_tr = pd.read_pickle('kaggle/prepo_train_met0.pkl')
met0_vl = pd.read_pickle('kaggle/prepo_val_met0.pkl')

## Compute mean meter reading by building by month on training data

In [7]:
from tsm.data_utils import compute_and_add_mean_by_a_and_b, only_add_mean_by_a_and_b

In [44]:
met0_tr, tr_means_by_bid_and_m = compute_and_add_mean_by_a_and_b(met0_tr, mean_of='meter_reading', a='building_id', b='dt_m')

In [45]:
met0_vl = only_add_mean_by_a_and_b(met0_vl, mean_of='meter_reading', a='building_id', b='dt_m', means_by_a_and_b=tr_means_by_bid_and_m)

#### Re-compress memory usage

In [14]:
from tsm.data_utils import compress_memory_usage

In [15]:
met0_tr = met0_tr.drop('timestamp', axis=1)

In [17]:
met0_tr, _ = compress_memory_usage(met0_tr)

In [18]:
met0_vl = met0_vl.drop('timestamp', axis=1)

In [19]:
met0_vl, _ = compress_memory_usage(met0_vl)

#### Save as pickle to save time later

In [20]:
met0_tr.to_pickle('kaggle/prepo_train_met0_bIDbyM_means.pkl')
met0_vl.to_pickle('kaggle/prepo_val_met0__bIDbyM_means.pkl')

In [20]:
import pickle

In [47]:
with open('kaggle/prepo_train_met0_tr_means_by_bid_and_m.pkl', 'wb') as f_out:
    pickle.dump(tr_means_by_bid_and_m, f_out)

#### Reloading: restart from here

In [13]:
met0_tr = pd.read_pickle('kaggle/prepo_train_met0_bIDbyM_means.pkl')
met0_vl = pd.read_pickle('kaggle/prepo_val_met0__bIDbyM_means.pkl')

In [50]:
with open('kaggle/prepo_train_met0_tr_means_by_bid_and_m.pkl', 'rb') as f_in:
    tr_means_by_bid_and_m = pickle.load(f_in)

## Baseline 1) LightGBM

In [24]:
import lightgbm as lgb

In [25]:
from tsm.eval_metrics import root_mean_squared_log_error

In [26]:
SEED = 0

In [27]:
lgb_reg_params = {'objective':'regression',  'boosting_type':'gbdt', 'metric':'rmse',
                  'n_jobs':-1, 'learning_rate':0.07, 'num_leaves': 2**8, 'max_depth':-1,
                  'tree_learner':'serial', 'colsample_bytree': 0.7, 'subsample_freq':1,
                  'subsample':0.5, 'max_bin':255, 'verbose':1, 'seed': SEED, 'early_stopping_rounds': 40}

In [21]:
xs = [x for x in list(met0_tr) if x not in ['timestamp', 'meter_reading']]
y = 'meter_reading'

In [22]:
met0_tr_x = met0_tr[xs]
met0_tr_y = met0_tr[y]
met0_vl_x = met0_vl[xs]
met0_vl_y = met0_vl[y]

In [62]:
%whos

Variable                          Type              Data/Info
-------------------------------------------------------------
SEED                              int               0
compute_and_add_mean_by_a_and_b   function          <function compute_and_add<...>_and_b at 0x7fb1a7f5bb70>
f_in                              BufferedReader    <_io.BufferedReader name=<...>_means_by_bid_and_m.pkl'>
f_out                             BufferedWriter    <_io.BufferedWriter name=<...>_means_by_bid_and_m.pkl'>
lgb                               module            <module 'lightgbm' from '<...>es/lightgbm/__init__.py'>
lgb_eval                          Dataset           <lightgbm.basic.Dataset object at 0x7fb16150ff60>
lgb_reg_params                    dict              n=15
lgb_train                         Dataset           <lightgbm.basic.Dataset object at 0x7fb16150f358>
met0_tr                           DataFrame                   building_id    <...>661397 rows x 59 columns]
met0_tr_x            

In [63]:
del met0_tr, met0_vl

In [28]:
lgb_train = lgb.Dataset(met0_tr_x, met0_tr_y)
lgb_eval = lgb.Dataset(met0_vl_x, met0_vl_y)
gbm_regress = lgb.train(lgb_reg_params, lgb_train, num_boost_round=2000, valid_sets=(lgb_train, lgb_eval),verbose_eval = 20)

/home/Ric/.local/lib/python3.5/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 119.759	valid_1's rmse: 124.142
[40]	training's rmse: 77.0525	valid_1's rmse: 85.5459
[60]	training's rmse: 69.6232	valid_1's rmse: 80.7385
[80]	training's rmse: 66.164	valid_1's rmse: 79.7223
[100]	training's rmse: 63.8183	valid_1's rmse: 78.8084
[120]	training's rmse: 61.7802	valid_1's rmse: 78.0157
[140]	training's rmse: 60.345	valid_1's rmse: 77.5614
[160]	training's rmse: 59.1499	valid_1's rmse: 77.4934
[180]	training's rmse: 58.2036	valid_1's rmse: 77.2936
[200]	training's rmse: 57.2973	valid_1's rmse: 77.3272
[220]	training's rmse: 56.3738	valid_1's rmse: 76.9794
[240]	training's rmse: 55.68	valid_1's rmse: 76.9387
[260]	training's rmse: 54.8786	valid_1's rmse: 76.9175
[280]	training's rmse: 54.2309	valid_1's rmse: 76.9433
Early stopping, best iteration is:
[252]	training's rmse: 55.2124	valid_1's rmse: 76.767


In [29]:
lgbm_tr_hat = (gbm_regress.predict(met0_tr_x, num_iteration=gbm_regress.best_iteration))
print('Training NRMLSE:', root_mean_squared_log_error(lgbm_tr_hat, met0_tr_y))

lgbm_vl_hat = (gbm_regress.predict(met0_vl_x, num_iteration=gbm_regress.best_iteration))
print('Validation NRMLSE:', root_mean_squared_log_error(lgbm_vl_hat, met0_vl_y))

/home/Ric/TSM/tsm/eval_metrics.py:5: RuntimeWarning: invalid value encountered in log
  return np.sqrt(np.sum(np.power(np.log(predicted + 1) - np.log(actual + 1), 2)) / len(actual))


Training NRMLSE: 0.6020488382795164
Validation NRMLSE: 0.6820828571711314
